# Enrich ToposText Annotation by Flair NER in Book 4

The notebook includes code designed to enhance the ToposText annotation through the utilization of the flair ner-large extraction.

The steps are:

- open the IOB-formatted ToposText annotation 'place' Class;
- open the output of the Flair ner-large annotation containing the LOC entities;
- for each Flair annotation, a validation process ensues, wherein it is ascertained whether it is either fully or partially present within the existing ToposText annotation;
- in cases where the Flair annotation is not already accounted for, it is integrated into the ToposText annotation.

The analysis determined that a total of 795 annotations identified by the Flair ner-large process were incorporated into the ToposText annotation.

In [ ]:
import pandas as pd

In [ ]:
## open the file containing ToposText annotations in Book 4 (18,664 rows)
ToposText_Book4 = pd.read_csv("/Users/u0154817/OneDrive - KU Leuven/Documents/KU Leuven/PhD project 'Greek Spaces in Roman Times'/Data_Extraction/Outputs/1.6.BIO_ToposText_Class_Book4.csv", delimiter=",")

In [ ]:
len(ToposText_Book4)

In [ ]:
## open the file containing the output of flair-ner large (18,664 entries)
NER_Flair_Book4 = pd.read_csv("/Users/u0154817/OneDrive - KU Leuven/Documents/KU Leuven/PhD project 'Greek Spaces in Roman Times'/Data_Extraction/Outputs/2.2.BIO_NER_Flair_spaCy_Book4.csv")

In [ ]:
len(NER_Flair_Book4)

In [ ]:
## select only the LOC annotations in flair ner-large

## create a copy of the Flair ner-large column
NER_Flair_Book4['BIO_Flair-large_copy'] = NER_Flair_Book4['BIO_Flair-large']

## the function trasnform all the annotations not including LOC into O
def update_values(value):
    if 'LOC' not in value:
        return 'O'
    return value

NER_Flair_Book4['BIO_Flair-large_copy'] = NER_Flair_Book4['BIO_Flair-large_copy'].apply(update_values)

In [ ]:
ToposText_Book4['flair_nerlarge'] = NER_Flair_Book4['BIO_Flair-large_copy']
ToposText_Book4['ToposText_update'] = ToposText_Book4['ToposText']

In [ ]:
for index, NER_annotation in enumerate(ToposText_Book4['flair_nerlarge']): ## for each token in flair ner-large
            
    if NER_annotation == 'B-LOC': ## for each B-LOC entity in flair ner-large
                
        if ToposText_Book4['ToposText_update'][index] == 'O': ## if no ToposText annotation exists
            
            if ToposText_Book4['flair_nerlarge'][index+1] != 'I-LOC': ## if B-LOC is not followed by I-LOC
                ToposText_Book4['ToposText_update'][index] = NER_annotation ## add the annotation
            
            else: ## if B-LOC is followed by I-LOC
                
                flag = False
                
                for n in range(1,100):
                    
                    if ToposText_Book4['flair_nerlarge'][index+n] == 'I-LOC': ## inside the multi-word LOC entity
                        
                        if len(ToposText_Book4['ToposText_update'][index+n]) > 1: ## ToposText annotated an entity in this span
                            flag = True
                            
                    else: break
                        
                if flag == False: ## no entity was predicted in the span
                    
                    ToposText_Book4['ToposText_update'][index] = NER_annotation ## add the B-LOC annotation
                    
                    for i in range(1,100):
                        if ToposText_Book4['flair_nerlarge'][index+i] == 'I-LOC':
                            ToposText_Book4['ToposText_update'][index+i] = 'I-LOC'
                        else : break

In [ ]:
ToposText_Book4['ToposText_update'].value_counts()